In [1]:
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('/home/koshirshov/koshirshov/RecBot')
import config
from pandas_utils import load_data
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
path = config.proceed_data_path
users_df = load_data(path + 'users_processed.pkl',)
items_df = load_data(path + 'items_processed.pkl',)
train_interactions = load_data(path + 'train_interactions.pkl',)
test_interactions = load_data(path + 'test_interactions.pkl',)
sample_submission = load_data(path + 'sample_submission_processed.pkl',)

In [3]:
#popular
from models import PopularRecommender
from recommendation_utils import create_recommendations_from_one_model, create_submission_file
from metrics import compute_metrics
pop_model = PopularRecommender(days=30)
pop_model.fit(train_interactions)
df_recommendations = create_recommendations_from_one_model(pop_model, test_interactions['user_id'].unique())
metrics = compute_metrics(train_interactions, test_interactions, df_recommendations)
print(metrics)

{'Precision@1': 0.07489781772115592, 'Recall@1': 0.04826541257304038, 'Precision@3': 0.07616463895594809, 'Recall@3': 0.1375204993719853, 'Precision@10': 0.03765387097545237, 'Recall@10': 0.2108414786587126, 'MAP@10': 0.10311451737903853, 'Novelty@10': 3.435862548188481}


In [4]:
class Rating_Datset(torch.utils.data.Dataset):
    def __init__(self, user_list, item_pos, item_neg):
        super(Rating_Datset, self).__init__()
        self.user_list = user_list
        self.item_pos = item_pos
        self.item_neg= item_neg

    def __len__(self):
        return len(self.user_list)

    def __getitem__(self, idx):
        user = self.user_list[idx]
        item_pos = self.item_pos[idx]
        item_neg = self.item_neg[idx]
        
        return (
            torch.tensor(user, dtype=torch.long),
            torch.tensor(item_pos, dtype=torch.long),
            torch.tensor(item_neg, dtype=torch.long)
            )
    

class NCF_Data(object):
    """
    Construct Dataset for NCF
    """
    def __init__(self, train_interactions, test_interactions, num_neg_train, num_neg_test, batch_size):
        self.num_neg_train = num_neg_train
        self.num_neg_test = num_neg_test
        self.batch_size = batch_size
        
        self.train_interactions, self.test_interactions = self._reindex(train_interactions, test_interactions)
        self.user_pool = set(self.train_interactions['user_id'])
        self.item_pool = set(self.train_interactions['item_id'])
        
    def _reindex(self, train_interactions, test_interactions):
        """
        Process dataset to reindex userID and itemID, also set rating as binary feedback
        """
        #ratings = ratings[ratings['watched_pct'] > 25]
        all_users = set(train_interactions['user_id'])
        train_interactions['quantity'] = 1
        good_users = train_interactions.groupby('user_id')['quantity'].sum()
        good_users = set(good_users[good_users >= 5].index)
        good_items = train_interactions.groupby(['item_id'])['quantity'].sum()
        good_items = set(good_items[good_items >= 10].index)
        
        train_interactions = train_interactions[train_interactions['user_id'].isin(good_users)]
        train_interactions = train_interactions[train_interactions['item_id'].isin(good_items)]
        
        self.users_not_in_model = all_users - set(train_interactions['user_id'])
        
        user_list = list(train_interactions['user_id'].drop_duplicates())
        self.user2id = {w: i for i, w in enumerate(user_list)}
        self.user2id_reverse = {w: i for i, w in enumerate(user_list)}
        
        item_list = list(train_interactions['item_id'].drop_duplicates())
        self.item2id = {w: i for i, w in enumerate(item_list)}
        self.item2id_reverse = {i: w for i, w in enumerate(item_list)}
        
        self.users_for_prediction_model = (set(train_interactions['user_id'])&set(test_interactions['user_id'])) - self.users_not_in_model
        self.users_for_prediction_popular = (set(test_interactions['user_id']) - self.users_for_prediction_model).union(self.users_not_in_model)
        self.users_for_prediction_model = [user for user in self.users_for_prediction_model]
        self.users_for_prediction_popular = [user for user in self.users_for_prediction_popular]
        
        train_interactions.loc[:,'user_id'] = train_interactions.loc[:,'user_id'].apply(lambda x: self.user2id[x])
        train_interactions.loc[:,'item_id'] = train_interactions.loc[:,'item_id'].apply(lambda x: self.item2id[x])
        train_interactions.loc[:,'rating'] = train_interactions.loc[:,'total_dur'].apply(lambda x: float(x >= 0))
        train_interactions = train_interactions[['user_id', 'item_id', 'rating', 'date']]
        
        test_interactions['rating'] = 1
        test_interactions = test_interactions[test_interactions['user_id'].isin(self.user2id.keys())]
        test_interactions = test_interactions[test_interactions['item_id'].isin(self.item2id.keys())]
        test_interactions = test_interactions.sample(30000)
        test_interactions.loc[:,'user_id'] = test_interactions.loc[:,'user_id'].apply(lambda x: self.user2id[x])
        test_interactions.loc[:,'item_id'] = test_interactions.loc[:,'item_id'].apply(lambda x: self.item2id[x])
        return train_interactions[['user_id', 'item_id', 'rating']], test_interactions[['user_id', 'item_id', 'rating']]
    
    def _leave_one_out(self, ratings):
        """
        leave-one-out evaluation protocol in paper https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf
        """
        ratings['rank_latest'] = ratings.groupby(['user_id'])['date'].rank(method='first', ascending=False)
        test = ratings.loc[ratings['rank_latest'] == 1]
        train = ratings.loc[ratings['rank_latest'] > 1]
        assert train['user_id'].nunique()==test['user_id'].nunique(), 'Not Match Train User with Test User'
        return train[['user_id', 'item_id', 'rating']], test[['user_id', 'item_id', 'rating']]
    
    def _negative_sampling(self, interactions, n):
        interact_status = (
            interactions.groupby('user_id')['item_id']
            .apply(set)
            .reset_index()
            .rename(columns={'item_id': 'interacted_items'}))
        interact_status['negative_samples'] = interact_status.loc[:,'interacted_items'].apply(lambda x: random.sample(self.item_pool - x, n))
        return interact_status[['user_id', 'negative_samples']]
    
    def get_train_instance(self):
        kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
        users, items_pos, items_neg = [], [], []
        negatives_train = self._negative_sampling(self.train_interactions, self.num_neg_train)
        train_interactions = pd.merge(self.train_interactions, negatives_train, on='user_id')
        self.negatives_test = self._negative_sampling(test_interactions, self.num_neg_test)     
        for row in train_interactions.itertuples():
            for i in range(self.num_neg_train):
                users.append(int(row.user_id))
                items_pos.append(int(row.item_id))
                items_neg.append(int(row.negative_samples[i]))
                
        dataset = Rating_Datset(
            user_list=users,
            item_pos=items_pos,
            item_neg=items_neg)
        
        return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True, **kwargs)
    
    def get_test_instance(self, ):
        kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
        users, items_pos, items_neg = [], [], []
        negatives_test = self._negative_sampling(self.test_interactions, self.num_neg_test)
        test_interactions = pd.merge(self.test_interactions, negatives_test, on='user_id')
        
        for row in test_interactions.itertuples():
            for i in range(self.num_neg_train):
                users.append(int(row.user_id))
                items_pos.append(int(row.item_id))
                items_neg.append(int(row.negative_samples[i]))
                
        dataset = Rating_Datset(
            user_list=users,
            item_pos=items_pos,
            item_neg=items_neg)
        
        return torch.utils.data.DataLoader(dataset, batch_size=self.num_neg_test, shuffle=True, **kwargs)

In [5]:
data = NCF_Data(train_interactions, test_interactions, num_neg_train=4, num_neg_test=25, batch_size=256)
train_loader = data.get_train_instance()
test_loader = data.get_test_instance()

## GMF

In [6]:
class Generalized_Matrix_Factorization(nn.Module):
    def __init__(self, num_users, num_items, factor_num):
        super(Generalized_Matrix_Factorization, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = factor_num
        
        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.affine_output = nn.Linear(in_features=self.factor_num, out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        nn.init.normal_(self.embedding_user.weight, std=0.01)
        nn.init.normal_(self.embedding_item.weight, std=0.01)
    
    def recommend(self, users, batch_size, n = config.num_recommendations, show_progress_bar = False):
        recs = list()
        user_count = len(users)
        
        for start in tqdm(
            range(0, user_count, batch_size),
            desc="predict from model, users:",
            disable = not show_progress_bar
        ):
            end = start + batch_size
            if end > user_count:
                end = user_count
                
            user_indices = torch.tensor(users[start:end]).to(device).repeat_interleave(self.num_items)
            item_indices = torch.tensor(range(self.num_items)).to(device).repeat((end-start))
            predict = self.forward(user_indices, item_indices).view((end-start), -1)
            predict = predict.topk(n)[1].detach().cpu()
            
            for rec in predict:
                recs.append(rec.tolist())
            
        return recs

In [7]:
model = Generalized_Matrix_Factorization(len(data.user2id), len(data.item2id), 64)
model = model.to(device)
model.init_weight()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [8]:
print(model)

Generalized_Matrix_Factorization(
  (embedding_user): Embedding(281156, 64)
  (embedding_item): Embedding(8250, 64)
  (affine_output): Linear(in_features=64, out_features=1, bias=True)
  (logistic): Sigmoid()
)


In [13]:
def evaluate_model(model, batch_size):
    start_time = time.time()
    model.eval()
    preds = model.recommend([data.user2id[user_id] for user_id in data.users_for_prediction_model], batch_size = batch_size)
    preds = [[data.item2id_reverse[item] for item in str_] for str_ in preds]
    
    df_recommendations = pd.DataFrame({'user_id': data.users_for_prediction_model})
    df_recommendations['item_id'] = preds
    df_recommendations = df_recommendations.explode('item_id')
    df_recommendations['rank'] = df_recommendations.groupby('user_id').cumcount() + 1
    
    df_recommendations_popular = create_recommendations_from_one_model(pop_model, data.users_for_prediction_popular)
    df_recommendations = pd.concat([df_recommendations, df_recommendations_popular], axis = 0)
    metrics = compute_metrics(train_interactions, test_interactions, df_recommendations)
    print("The time for evaluate_model is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
    return metrics

def calculate_test_loss(model, test_loader, loss_function):
    start_time = time.time()
    loss_accum = 0
    model.eval()
    for i, (user, item, label) in enumerate(test_loader):
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)
        prediction = model(user, item).squeeze()
        loss = loss_function(prediction, label)
        loss_accum += loss
    print("The time for test is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
    return loss_accum

def train_model(model, train_loader, test_loader, loss_function, optimizer, scheduler, num_epochs):
    loss_history_train = []
    loss_history_test = []
    test_metrics_history = []
    for epoch in range(1, num_epochs + 1):
        print(f"epoch: {epoch}")
        start_time = time.time()
        loss_accum = 0
        train_loader = data.get_train_instance() # refresh negative samples
        model.train() # Enter train mode
        for (user, item, label) in tqdm(train_loader):
            user = user.to(device)
            item = item.to(device)
            label = label.to(device)  
            prediction = model(user, item).squeeze()
            loss = loss_function(prediction, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_accum += loss
            
        loss_accum = loss_accum
        loss_history_train.append(loss_accum)
        test_loss = calculate_test_loss(model, test_loader, loss_function)
        loss_history_test.append(test_loss)
        metrics = evaluate_model(model, batch_size=64)
        test_metrics_history.append(metrics)
        
        torch.cuda.empty_cache()
        
        if scheduler:
            scheduler.step()
        print(f"train_loss: {loss_accum} \ntest_loss: {test_loss} \nmetrics_on_test_set: \n{metrics}")
        print("The time elapse of epoch {:03d}".format(epoch) + " is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
        print("\n")
    return loss_history_train, loss_history_test, test_metrics_history
loss_history_train, loss_history_test, test_metrics_history = train_model(model, train_loader, test_loader, loss_function, optimizer, scheduler, 10)

epoch: 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152657/152657 [19:27<00:00, 130.79it/s]


The time for test is: 00: 00: 24
The time for evaluate_model is: 00: 00: 52
train_loss: 19035.51953125 
test_loss: 34641.7578125 
metrics_on_test_set: 
{'Precision@1': 0.06775103377393217, 'Recall@1': 0.04483039494926464, 'Precision@3': 0.06544645489239986, 'Recall@3': 0.12223095780492316, 'Precision@10': 0.03286624279943591, 'Recall@10': 0.18940929710410762, 'MAP@10': 0.09293149691786595, 'Novelty@10': 3.697623317192161}
The time elapse of epoch 001 is: 00: 21: 52


epoch: 2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152657/152657 [19:03<00:00, 133.48it/s]


The time for test is: 00: 00: 21
The time for evaluate_model is: 00: 00: 46
train_loss: 22028.90625 
test_loss: 19967.1640625 
metrics_on_test_set: 
{'Precision@1': 0.07118698759471281, 'Recall@1': 0.04635837305881009, 'Precision@3': 0.0683047700998319, 'Recall@3': 0.12610507667865228, 'Precision@10': 0.034750340607596145, 'Recall@10': 0.19840607569504742, 'MAP@10': 0.09646168536512593, 'Novelty@10': 3.6179461263157755}
The time elapse of epoch 002 is: 00: 21: 15


epoch: 3


...

The time for test is: 00: 00: 23
The time for evaluate_model is: 00: 00: 48
train_loss: 24736.955078125 
test_loss: 15085.8173828125 
metrics_on_test_set: 
{'Precision@1': 0.06993809307550733, 'Recall@1': 0.04587452857451676, 'Precision@3': 0.06787253706846412, 'Recall@3': 0.12547013849111388, 'Precision@10': 0.03490092501852427, 'Recall@10': 0.19846431906753123, 'MAP@10': 0.09598075133716916, 'Novelty@10': 3.594114421349478}
The time elapse of epoch 003 is: 00: 43: 49


epoch: 4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152657/152657 [19:15<00:00, 132.07it/s]


The time for test is: 00: 00: 22
The time for evaluate_model is: 00: 00: 48
train_loss: 26978.453125 
test_loss: 12048.2705078125 
metrics_on_test_set: 
{'Precision@1': 0.07349953390539475, 'Recall@1': 0.0475177754415706, 'Precision@3': 0.07104357387001936, 'Recall@3': 0.12989589394069725, 'Precision@10': 0.03630817219207879, 'Recall@10': 0.20468962928054554, 'MAP@10': 0.09927905510724082, 'Novelty@10': 3.563676687238595}
The time elapse of epoch 004 is: 00: 21: 29


epoch: 5


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152657/152657 [19:32<00:00, 130.21it/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Multi_Layer_Perceptron

In [6]:
class Multi_Layer_Perceptron(nn.Module):
    def __init__(self, num_users, num_items, factor_num):
        super(Multi_Layer_Perceptron, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = factor_num
        self.layers = [factor_num*2,32,16]

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
            #vector = nn.BatchNorm1d()(vector)
            vector = nn.Dropout(p=0.25)(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        nn.init.normal_(self.embedding_user.weight, std=0.01)
        nn.init.normal_(self.embedding_item.weight, std=0.01)
        
    def recommend(self, users, batch_size, n = config.num_recommendations, show_progress_bar = False):
        recs = list()
        user_count = len(users)
        
        for start in tqdm(
            range(0, user_count, batch_size),
            desc="predict from model, users:",
            disable = not show_progress_bar
        ):
            end = start + batch_size
            if end > user_count:
                end = user_count
                
            user_indices = torch.tensor(users[start:end]).to(device).repeat_interleave(self.num_items)
            item_indices = torch.tensor(range(self.num_items)).to(device).repeat((end-start))
            predict = self.forward(user_indices, item_indices).view((end-start), -1)
            predict = predict.topk(n)[1].detach().cpu()
            
            for rec in predict:
                recs.append(rec.tolist())
            
        return recs

In [7]:
model = Multi_Layer_Perceptron(len(data.user2id), len(data.item2id), 64)
model = model.to(device)
model.init_weight()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [8]:
print(model)

Multi_Layer_Perceptron(
  (embedding_user): Embedding(281156, 64)
  (embedding_item): Embedding(8250, 64)
  (fc_layers): ModuleList(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
  )
  (affine_output): Linear(in_features=16, out_features=1, bias=True)
  (logistic): Sigmoid()
)


In [9]:
def evaluate_model(model, batch_size):
    start_time = time.time()
    model.eval()
    preds = model.recommend([data.user2id[user_id] for user_id in data.users_for_prediction_model], batch_size = batch_size)
    preds = [[data.item2id_reverse[item] for item in str_] for str_ in preds]
    
    df_recommendations = pd.DataFrame({'user_id': data.users_for_prediction_model})
    df_recommendations['item_id'] = preds
    df_recommendations = df_recommendations.explode('item_id')
    df_recommendations['rank'] = df_recommendations.groupby('user_id').cumcount() + 1
    
    df_recommendations_popular = create_recommendations_from_one_model(pop_model, data.users_for_prediction_popular)
    df_recommendations = pd.concat([df_recommendations, df_recommendations_popular], axis = 0)
    metrics = compute_metrics(train_interactions, test_interactions, df_recommendations)
    print("The time for evaluate_model is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
    return metrics

def calculate_test_loss(model, test_loader, loss_function):
    start_time = time.time()
    loss_accum = 0
    model.eval()
    for i, (user, item, label) in enumerate(test_loader):
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)
        prediction = model(user, item).squeeze()
        loss = loss_function(prediction, label)
        loss_accum += loss
    print("The time for test is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
    return loss_accum

def train_model(model, train_loader, test_loader, loss_function, optimizer, scheduler, num_epochs):
    loss_history_train = []
    loss_history_test = []
    test_metrics_history = []
    for epoch in range(1, num_epochs + 1):
        print(f"epoch: {epoch}")
        start_time = time.time()
        loss_accum = 0
        train_loader = data.get_train_instance() # refresh negative samples
        model.train() # Enter train mode
        for (user, item, label) in tqdm(train_loader):
            user = user.to(device)
            item = item.to(device)
            label = label.to(device)  
            prediction = model(user, item).squeeze()
            loss = loss_function(prediction, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_accum += loss
            
        loss_accum = loss_accum
        loss_history_train.append(loss_accum)
        test_loss = calculate_test_loss(model, test_loader, loss_function)
        loss_history_test.append(test_loss)
        metrics = evaluate_model(model, batch_size=64)
        test_metrics_history.append(metrics)
        
        torch.cuda.empty_cache()
        
        if scheduler:
            scheduler.step()
        print(f"train_loss: {loss_accum} \ntest_loss: {test_loss} \nmetrics_on_test_set: \n{metrics}")
        print("The time elapse of epoch {:03d}".format(epoch) + " is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
        print("\n")
    return loss_history_train, loss_history_test, test_metrics_history
loss_history_train, loss_history_test, test_metrics_history = train_model(model, train_loader, test_loader, loss_function, optimizer, scheduler, 10)

epoch: 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152657/152657 [21:15<00:00, 119.71it/s]


The time for test is: 00: 00: 31
The time for evaluate_model is: 00: 00: 52
train_loss: 32706.845703125 
test_loss: 94401.5546875 
metrics_on_test_set: 
{'Precision@1': 0.06763749790854985, 'Recall@1': 0.044795864948753365, 'Precision@3': 0.06605197950777224, 'Recall@3': 0.12311731909516226, 'Precision@10': 0.03389344360255277, 'Recall@10': 0.19423844562481377, 'MAP@10': 0.09394437959415965, 'Novelty@10': 3.640449518629348}
The time elapse of epoch 001 is: 00: 23: 47


epoch: 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152657/152657 [9:15:40<00:00,  4.58it/s]


The time for test is: 00: 00: 34
The time for evaluate_model is: 00: 00: 56
train_loss: 29794.197265625 
test_loss: 134229.953125 
metrics_on_test_set: 
{'Precision@1': 0.06633482324258431, 'Recall@1': 0.044171831389755126, 'Precision@3': 0.06483296284787789, 'Recall@3': 0.12144586626143039, 'Precision@10': 0.03340762961015369, 'Recall@10': 0.19193624566621187, 'MAP@10': 0.09269438637117772, 'Novelty@10': 3.673500529396491}
The time elapse of epoch 002 is: 09: 18: 15


epoch: 3


 13%|███████████████▍                                                                                                         | 19478/152657 [02:52<19:38, 112.98it/s]


KeyboardInterrupt: 

## NeuMF

In [6]:
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, factor_num_mf, factor_num_mlp):
        super(NeuMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num_mf = factor_num_mf
        self.factor_num_mlp =  factor_num_mlp
        self.layers = [2*factor_num_mlp, 32, 16]
        self.dropout = 0.2
        self.reg_1=0.0003
        self.reg_2=0.0003
        self.embedding_user_mlp = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mf)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Dropout(p=self.dropout))
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
            self.fc_layers.append(nn.ReLU())

        self.affine_output = nn.Linear(in_features=self.layers[-1] + self.factor_num_mf, out_features=1)
        self.init_weight()

    def init_weight(self):
        nn.init.normal_(self.embedding_user_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_user_mf.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mf.weight, std=0.01)
        
        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                
        nn.init.xavier_uniform_(self.affine_output.weight)

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user_indices, item_indices):
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)

        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)  # the concat latent vector
        mf_vector = torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        return logits
    
    def recommend(self, users, batch_size, n = config.num_recommendations, show_progress_bar = False):
        recs = list()
        user_count = len(users)
        
        for start in tqdm(
            range(0, user_count, batch_size),
            desc="predict from model, users:",
            disable = not show_progress_bar
        ):
            end = start + batch_size
            if end > user_count:
                end = user_count
                
            user_indices = torch.tensor(users[start:end]).to(device).repeat_interleave(self.num_items)
            item_indices = torch.tensor(range(self.num_items)).to(device).repeat((end-start))
            predict = self.forward(user_indices, item_indices).view((end-start), -1)
            predict = predict.topk(n)[1].detach().cpu()
            
            for rec in predict:
                recs.append(rec.tolist())
            
        return recs

In [7]:
model = NeuMF(len(data.user2id), len(data.item2id), factor_num_mf=32, factor_num_mlp=32)
model = model.to(device)
model.init_weight()
optimizer = optim.Adam(model.parameters(), 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [8]:
print(model)

NeuMF(
  (embedding_user_mlp): Embedding(281156, 32)
  (embedding_item_mlp): Embedding(8250, 32)
  (embedding_user_mf): Embedding(281156, 32)
  (embedding_item_mf): Embedding(8250, 32)
  (fc_layers): ModuleList(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
  )
  (affine_output): Linear(in_features=48, out_features=1, bias=True)
)


In [11]:
def evaluate_model(model, batch_size):
    start_time = time.time()
    model.eval()
    preds = model.recommend([data.user2id[user_id] for user_id in data.users_for_prediction_model], batch_size = batch_size)
    preds = [[data.item2id_reverse[item] for item in str_] for str_ in preds]
    
    df_recommendations = pd.DataFrame({'user_id': data.users_for_prediction_model})
    df_recommendations['item_id'] = preds
    df_recommendations = df_recommendations.explode('item_id')
    df_recommendations['rank'] = df_recommendations.groupby('user_id').cumcount() + 1
    
    df_recommendations_popular = create_recommendations_from_one_model(pop_model, data.users_for_prediction_popular)
    df_recommendations = pd.concat([df_recommendations, df_recommendations_popular], axis = 0)
    metrics = compute_metrics(train_interactions, test_interactions, df_recommendations)
    print("The time for evaluate_model is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
    return metrics

def calculate_test_loss(model, test_loader):
    start_time = time.time()
    loss_type = "BPR"
    loss_accum = 0
    model.eval()
    for i, (user, item_pos, item_neg) in enumerate(test_loader):
        user = user.to(device)
        item_pos = item_pos.to(device)
        item_neg = item_neg.to(device)
        prediction_pos = model(user, item_pos).squeeze()
        prediction_neg = model(user, item_neg).squeeze()
        if loss_type == 'BPR':
            loss = -((prediction_pos - prediction_neg).sigmoid() + 1e-24).log().sum()
        elif loss_type == 'HL': #hinge_loss
            loss = torch.clamp(1 - (prediction_pos - prediction_neg) * label, min=0).sum()
        elif loss_type == 'TL':
            loss = (prediction_neg - prediction_pos).sigmoid().mean() + prediction_neg.pow(2).sigmoid().mean()
        loss_accum += loss
    print("The time for test is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
    return loss_accum


def train_model(model, train_loader, test_loader, optimizer, scheduler, num_epochs, loss_type = "BPR"):
    loss_history_train = []
    loss_history_test = []
    test_metrics_history = []
    loss_type = "BPR"
    for epoch in range(1, num_epochs + 1):
        print(f"epoch: {epoch}")
        start_time = time.time()
        loss_accum = 0
        train_loader = data.get_train_instance() # refresh negative samples
        model.train() # Enter train mode
        for (user, item_pos, item_neg) in tqdm(train_loader):
            user = user.to(device)
            item_pos = item_pos.to(device)
            item_neg = item_neg.to(device)
            prediction_pos = model(user, item_pos).squeeze()
            prediction_neg = model(user, item_neg).squeeze()
            if loss_type == 'BPR':
                loss = -((prediction_pos - prediction_neg).sigmoid() + 1e-24).log().sum()
            elif loss_type == 'HL':
                loss = torch.clamp(1 - (prediction_pos - prediction_neg) * label, min=0).sum()
            elif loss_type == 'TL':
                loss = (prediction_neg - prediction_pos).sigmoid().mean() + prediction_neg.pow(2).sigmoid().mean()
                
            loss += model.reg_1 * (model.embedding_item_mf.weight.norm(p=1) + model.embedding_user_mf.weight.norm(p=1))
            loss += model.reg_2 * (model.embedding_item_mlp.weight.norm(p=1) + model.embedding_user_mlp.weight.norm(p=1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_accum += loss
            
        loss_accum = loss_accum
        loss_history_train.append(loss_accum)
        test_loss = calculate_test_loss(model, test_loader)
        loss_history_test.append(test_loss)
        metrics = evaluate_model(model, batch_size=64)
        test_metrics_history.append(metrics)
        
        torch.cuda.empty_cache()
        
        if scheduler:
            scheduler.step()
        print(f"train_loss: {loss_accum} \ntest_loss: {test_loss}\nmetrics_on_test_set: \n{metrics}")
        print("The time elapse of epoch {:03d}".format(epoch) + " is: " + time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)))
        print("\n")
    return loss_history_train, loss_history_test, test_metrics_history
loss_history_train, loss_history_test, test_metrics_history = train_model(model, train_loader, test_loader, optimizer, scheduler, 10)

epoch: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61063/61063 [13:04<00:00, 77.81it/s]


The time for test is: 00: 00: 05
The time for evaluate_model is: 00: 00: 51
train_loss: 4253767.5 
test_loss: 36232.01953125
metrics_on_test_set: 
{'Precision@1': 0.07774218992757607, 'Recall@1': 0.04989046698821061, 'Precision@3': 0.07557106548430019, 'Recall@3': 0.13678386417309818, 'Precision@10': 0.03774828501087554, 'Recall@10': 0.21115707218306118, 'MAP@10': 0.10374922550530463, 'Novelty@10': 3.4559851433977307}
The time elapse of epoch 001 is: 00: 15: 08


epoch: 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61063/61063 [13:01<00:00, 78.13it/s]


The time for test is: 00: 00: 06
The time for evaluate_model is: 00: 00: 52
train_loss: 3618861.25 
test_loss: 35490.10546875
metrics_on_test_set: 
{'Precision@1': 0.07701914573224658, 'Recall@1': 0.04954284425073445, 'Precision@3': 0.07303941487200326, 'Recall@3': 0.13334000931490192, 'Precision@10': 0.03753077419509046, 'Recall@10': 0.210665921254339, 'MAP@10': 0.1028588590604676, 'Novelty@10': 3.4745896735383885}
The time elapse of epoch 002 is: 00: 15: 04


epoch: 3


...

The time for test is: 00: 00: 06
The time for evaluate_model is: 00: 00: 49
train_loss: 3149312.75 
test_loss: 37029.15625
metrics_on_test_set: 
{'Precision@1': 0.07836962497310992, 'Recall@1': 0.049843168938969146, 'Precision@3': 0.07344376190134727, 'Recall@3': 0.1332448032446092, 'Precision@10': 0.037768601955207114, 'Recall@10': 0.21206831567820894, 'MAP@10': 0.10332783304768256, 'Novelty@10': 3.474939954039873}
The time elapse of epoch 003 is: 00: 14: 55


epoch: 4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61063/61063 [13:09<00:00, 77.33it/s]


The time for test is: 00: 00: 07
The time for evaluate_model is: 00: 02: 02
train_loss: 2858805.5 
test_loss: 37413.34375
metrics_on_test_set: 
{'Precision@1': 0.07469464827784018, 'Recall@1': 0.04807109440385605, 'Precision@3': 0.07526431946203918, 'Recall@3': 0.1363453021167678, 'Precision@10': 0.03759112747089896, 'Recall@10': 0.2108726749249067, 'MAP@10': 0.10252421316046631, 'Novelty@10': 3.483877823511584}
The time elapse of epoch 004 is: 00: 16: 21


epoch: 5


 38%|██████████████████████████████████████████████▋                                                                            | 23159/61063 [36:04<59:02, 10.70it/s]


KeyboardInterrupt: 